In [ ]:
!pip install sacrebleu
!pip install -U scikit-learn
!pip install transformers

import re
import sacrebleu

from sklearn.model_selection import train_test_split

import pandas as pd

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(65100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(65100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
file_path_train = '/content/drive/MyDrive/data/train.csv'
file_path_val = '/content/drive/MyDrive/data/validation.csv'
train = pd.read_csv(file_path_train)
val = pd.read_csv(file_path_val)

In [ ]:
# Prepare for forward evaluation from english to chinese of pretrained model
prefix = 'translate to zh: '

predictions, references = [], []

for index, row in val.iterrows():
    print("Row: ", str(index))
    print("Translating ", row["en"])
    src_text = prefix + row['en']
    input_ids = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
    generated_tokens = model.generate(input_ids)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    predictions.append(translation)
    references.append(row['zh'])

references = [[reference] for reference in references]

pattern = r'([\u4e00-\u9fff])\.$'
punctuation_mapping = {
    ',': '，',
    '?': '？',
    '!': '！',
    ':': '：',
    ';': '；',
    '"': '”',
    "'": '’',
    '(': '（',
    ')': '）',
    '[': '【',
    ']': '】',
    '{': '｛',
    '}': '｝'
}

processed_predictions = []
processed_references = []
for pred in predictions:
    pred = pred.replace(" ", "")
    for eng_punc, chi_punc in punctuation_mapping.items():
        pred = pred.replace(eng_punc, chi_punc)
    pred = re.sub(pattern, r'\1。', pred)
    processed_predictions.append(pred)

for ref in references:
    ref = ref[0].replace(" ", "")
    for eng_punc, chi_punc in punctuation_mapping.items():
        ref = ref.replace(eng_punc, chi_punc)
    ref = re.sub(pattern, r'\1。', ref)
    processed_references.append([ref])

def process_string(text):
    return replace_punctuation_with_zh(remove_spaces(text))

def remove_spaces(text):
    return ''.join(text.split())

# Replacing en punctuation with the zh counterparts
def replace_punctuation_with_zh(text):
    # Define a mapping of English punctuation to Chinese punctuation
    punctuation_mapping = {
        ',': '，',
        '.': '。',
        '?': '？',
        '!': '！',
        ':': '：',
        ';': '；',
        '"': '”',
        "'": '’',
        '(': '（',
        ')': '）',
        '[': '【',
        ']': '】',
        '{': '｛',
        '}': '｝'
    }

    # Replace each English punctuation with its Chinese counterpart
    for eng_punc, chi_punc in punctuation_mapping.items():
        text = text.replace(eng_punc, chi_punc)

    return text

processed_predictions = [process_string(sent) for sent in predictions]
processed_references = [process_string(sent[0]) for sent in references]

bleu_score_no_smoothing = sacrebleu.corpus_bleu(processed_predictions, processed_references, tokenize='zh')
bleu_score_add_1 = sacrebleu.corpus_bleu(processed_predictions, processed_references, tokenize='zh', smooth_method="add-k", smooth_value=1)

print(f"BLEU Score without smoothing: BLEU = {bleu_score_no_smoothing}")
print(f"BLEU Score with smoothing: BLEU = {bleu_score_add_1}")

Row:  0
Translating  Last year I showed these two slides so that  demonstrate that the arctic ice cap,  which for most of the last three million years  has been the size of the lower 48 states,  has shrunk by 40 percent.
Row:  1
Translating  But this understates the seriousness of this particular problem  because it doesn't show the thickness of the ice.
Row:  2
Translating  The arctic ice cap is, in a sense,  the beating heart of the global climate system.
Row:  3
Translating  It expands in winter and contracts in summer.
Row:  4
Translating  The next slide I show you will be  a rapid fast-forward of what's happened over the last 25 years.
Row:  5
Translating  The permanent ice is marked in red.
Row:  6
Translating  As you see, it expands to the dark blue --  that's the annual ice in winter,  and it contracts in summer.
Row:  7
Translating  The so-called permanent ice, five years old or older,  you can see is almost like blood,  spilling out of the body here.
Row:  8
Translating  In 2

In [ ]:
# Prepare for backwards evaluation from chinese to english of pretrained model
prefix = 'translate to en: '

predictions_back, references_back = [], []

for index, row in val.iterrows():
    print("Row: ", str(index))
    print("Translating ", row["zh"])
    src_text = prefix + row['zh']
    input_ids = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
    generated_tokens = model.generate(input_ids)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    predictions_back.append(translation)
    references_back.append(row['en'])

references_back = [[reference] for reference in references_back]

pattern = r'([a-zA-Z])\.$'
punctuation_mapping = {
    ',': '，',
    '?': '？',
    '!': '！',
    ':': '：',
    ';': '；',
    '"': '”',
    "'": '’',
    '(': '（',
    ')': '）',
    '[': '【',
    ']': '】',
    '{': '｛',
    '}': '｝'
}

processed_predictions_back = []
processed_references_back = []
for pred in predictions_back:
    pred = pred.replace(" ", "")
    for chi_punc, eng_punc in punctuation_mapping.items():
        pred = pred.replace(chi_punc, eng_punc)
    pred = re.sub(pattern, r'\1。', pred)
    processed_predictions_back.append(pred)

for ref in references_back:
    ref = ref[0].replace(" ", "")
    for chi_punc, eng_punc in punctuation_mapping.items():
        ref = ref.replace(chi_punc, eng_punc)
    ref = re.sub(pattern, r'\1。', ref)
    processed_references_back.append([ref])

def process_string(text):
    return replace_punctuation_with_en(remove_spaces(text))

def remove_spaces(text):
    return ''.join(text.split())

# Replacing en punctuation with the zh counterparts
def replace_punctuation_with_en(text):
    # Define a mapping of English punctuation to Chinese punctuation
    punctuation_mapping = {
        '，': ',',
        '。': '.',
        '？': '?',
        '！': '!',
        '：': ':',
        '；': ';',
        '”': '"',
        '’': "'",
        '（': '(',
        '）': ')',
        '【': '[',
        '】': ']',
        '｛': '{',
        '｝': '}'
    }

    # Replace each English punctuation with its Chinese counterpart
    for chi_punc, eng_punc in punctuation_mapping.items():
        text = text.replace(chi_punc, eng_punc)

    return text

processed_predictions_back = [process_string(sent) for sent in predictions_back]
processed_references_back = [process_string(sent[0]) for sent in references_back]

bleu_score_no_smoothing_back = sacrebleu.corpus_bleu(processed_predictions_back, processed_references_back)
bleu_score_add_1_back = sacrebleu.corpus_bleu(processed_predictions_back, processed_references_back, smooth_method="add-k", smooth_value=1)

print(f"BLEU Score without smoothing: BLEU = {bleu_score_no_smoothing_back}")
print(f"BLEU Score with smoothing: BLEU = {bleu_score_add_1_back}")

Row:  0
Translating  去年我给各位展示了两个 关于北极冰帽的演示 在过去三百万年中 其面积由相当于美国南方48州面积总和 缩减了40%
Row:  1
Translating  但这些没能完全说明这个问题的严重性 因为这没有表示出冰帽的厚度
Row:  2
Translating  感觉上，北极冰帽 就好象全球气候系统中跳动的心脏
Row:  3
Translating  冬天心脏舒张，夏天心脏收缩
Row:  4
Translating  下面我要展示的是 在过去25年里的极剧变化
Row:  5
Translating  红色的是永冻冰
Row:  6
Translating  你看，它正在变成深蓝色 这是每年冬天形成的年度冰 在夏天永冻冰收缩
Row:  7
Translating  所谓的“永冻”，是指形成五年或更久的冰 你看，这也像血液一样 输送到身体各部位
Row:  8
Translating  在25年的时间里，它从这里，到了这里
Row:  9
Translating  值得注意的是 温室效应使得北冰洋周围的冻土层受热 而这里有大量被冻封的碳 解冻时，微生物降解碳形成甲烷
Row:  10
Translating  如果突破顶点，温室气体排放量 将是现有大气层中的全球温室污染总量
Row:  11
Translating  在阿拉斯加的一些浅湖里 已经可以看到水中探头的沼气泡
Row:  12
Translating  去年冬天，University of Alaska的Katey Walter教授 结队去了一个浅湖
Row:  13
Translating  哇（开心而又惊叹滴笑） 戈尔：她很好，我们怎么样呢？
Row:  14
Translating  有一个原因，北方沉积的大量热能 加热了格陵兰岛
Row:  15
Translating  这是一条每年融化的河
Row:  16
Translating  但流量却比往年都要大
Row:  17
Translating  这是格陵兰岛西南的Kangerlussuaq河
Row:  18
Translating  如果你想了解陆地上的冰块融化 如何使得海平面上升 这里就是它的入海口
Row:  19
Translating  这里的流量正在急速上升
Row:  20
Tran

In [ ]:
train_df, _ = train_test_split(train, train_size=0.05, random_state=42)
# train_df

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = row['en']
        target_text = row['zh']

        input_encoding = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        target_encoding = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }


# Define your optimizer and other training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
batch_size = 32
epochs = 1
max_length = 32  # Define your desired max sequence length
consistency_weight = 0.1

# Create DataLoader objects
train_dataset = MyDataset(train_df, tokenizer, max_length)
val_dataset = MyDataset(val, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import gensim.downloader as api
word2vec_model = api.load('word2vec-google-news-300')
!pip install pot

[==================================================] 100.0% 1662.8/1662.8MB downloaded
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
from gensim.models import KeyedVectors
word2vec_model.save('word2vec_model.bin')
word2vec_model = KeyedVectors.load('word2vec_model.bin')

In [ ]:
!pip install torchsort
from torchsort import soft_rank

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 725.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 981.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1

In [ ]:
import torch
from torch.nn.functional import cosine_similarity

def translate(model, tokenizer, text, prefix):
    src_text = prefix + text
    input_ids = tokenizer(src_text, return_tensors="pt").to(device)
    generated_tokens = model.generate(**input_ids)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]  # Access the first element of the list
    return result

def corrcoef(target, pred):
    # np.corrcoef in torch from @mdo
    # https://forum.numer.ai/t/custom-loss-functions-for-xgboost-using-pytorch/960
    pred_n = pred - pred.mean()
    target_n = target - target.mean()
    pred_n = pred_n / pred_n.norm()
    target_n = target_n / target_n.norm()
    return 1 - (pred_n * target_n).sum()


def spearman(
    target,
    pred,
    regularization="l2",
    regularization_strength=1.0,
):
    # fast_soft_sort uses 1-based indexing, divide by len to compute percentage of rank
    pred = soft_rank(
        pred,
        regularization=regularization,
        regularization_strength=regularization_strength,
    )
    return corrcoef(target, pred / pred.shape[-1])


def self_consistency_loss(original_text, model, tokenizer):
    # Translate to Chinese
    translated_text = translate(model, tokenizer, original_text, 'translate to zh: ')

    # Translate back to English
    back_translated_text = translate(model, tokenizer, translated_text, 'translate to en: ')

    # Encoded texts
    encoded_input = tokenizer(original_text, return_tensors="pt").to(device)
    encoded_back_translated_input = tokenizer(back_translated_text, return_tensors="pt").to(device)

    # Calculate cosine similarity between original and back-translated text embeddings
    original_embedding = model.get_encoder()(**encoded_input).last_hidden_state.mean(dim=1)
    back_translated_embedding = model.get_encoder()(**encoded_back_translated_input).last_hidden_state.mean(dim=1)
    # loss = 1 - cosine_similarity(original_embedding, back_translated_embedding)

    # uncomment to try word movers distance
    # loss = torch.tensor(word2vec_model.wmdistance(translated_text, back_translated_text))


    # uncomment to try spearman correlation coefficient
    s = spearman(original_embedding, back_translated_embedding, regularization_strength=1e-2)
    loss = torch.tensor(s.item())

    return loss

In [ ]:
import os

def save_checkpoint(model, optimizer, epoch, batch_index, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'batch_index': batch_index,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch}, batch {batch_index}")

# Define the checkpoint path
checkpoint_dir = "/content/drive/MyDrive/checkpoints" # TO CHANGE
os.makedirs(checkpoint_dir, exist_ok=True)

def load_checkpoint(checkpoint_path, model, optimizer):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    batch_index = checkpoint['batch_index']
    return epoch, batch_index

In [ ]:
latest_checkpoint_path = os.path.join(checkpoint_dir, "wmd.pt")
os.path.exists(latest_checkpoint_path)

False

In [ ]:
import time

# Check if a checkpoint exists
latest_checkpoint_path = os.path.join(checkpoint_dir, "wmd.pt")
print(latest_checkpoint_path)
if os.path.exists(latest_checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(latest_checkpoint_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    start_batch_index = checkpoint['batch_index'] + 1

    print(f"Restarting from {start_epoch} and batch {start_batch_index}")
else:
    start_epoch = 0
    start_batch_index = 0

num_batches = len(train_loader)
start_time = time.time()
print(f"Total number of batches: {num_batches}")

'''
# Define the range of batches to train
start_batch = 700
end_batch = 723
'''

for epoch in range(start_epoch, epochs):
    model.train()
    epoch_start_time = time.time()
    batch_number = 0
    for batch_index, batch in enumerate(train_loader, start=start_batch_index):
        batch_number += 1
        if batch_number < start_batch_index:
            continue  # Skip batches before the start_batch

        print("Batch number: ", batch_number)
        batch_start_time = time.time()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        total_loss = outputs.loss

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        # if batch_number % 100 == 0:
        if batch_number % 50 == 0:
            # Save checkpoint
            save_checkpoint(model, optimizer, epoch, batch_number, latest_checkpoint_path)

            # Time:
            curr_time = time.time()
            remaining_duration = ((curr_time - start_time) / (batch_number)) * (num_batches - batch_number)
            print(f"Batch number {batch_number}: Epoch {epoch}, Batch Loss: {total_loss.item()}, Estimated Time Left: {remaining_duration}")

    # Reset the start_batch_index after completing an epoch
    start_batch_index = 0

# Save the fine-tuned model
save_checkpoint(model, optimizer, epoch, batch_number, latest_checkpoint_path)

/content/drive/MyDrive/checkpoints/wmd.pt
Total number of batches: 362
Batch number:  1
Batch number:  2
Batch number:  3
Batch number:  4
Batch number:  5
Batch number:  6
Batch number:  7
Batch number:  8
Batch number:  9
Batch number:  10
Batch number:  11
Batch number:  12
Batch number:  13
Batch number:  14
Batch number:  15
Batch number:  16
Batch number:  17
Batch number:  18
Batch number:  19
Batch number:  20
Batch number:  21
Batch number:  22
Batch number:  23
Batch number:  24
Batch number:  25
Batch number:  26
Batch number:  27
Batch number:  28
Batch number:  29
Batch number:  30
Batch number:  31
Batch number:  32
Batch number:  33
Batch number:  34
Batch number:  35
Batch number:  36
Batch number:  37
Batch number:  38
Batch number:  39
Batch number:  40
Batch number:  41
Batch number:  42
Batch number:  43
Batch number:  44
Batch number:  45
Batch number:  46
Batch number:  47
Batch number:  48
Batch number:  49
Batch number:  50
Checkpoint saved at epoch 0, batch 50


In [ ]:
# Prepare for forward evaluation from english to chinese of pretrained model
prefix = 'translate to zh: '

predictions_tuned, references_tuned = [], []

for index, row in val.iterrows():
    print("Row: ", str(index))
    print("Translating ", row["en"])
    src_text = prefix + row['en']
    input_ids = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
    generated_tokens = model.generate(input_ids)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    predictions_tuned.append(translation)
    references_tuned.append(row['zh'])

references_tuned = [[reference] for reference in references_tuned]

pattern = r'([\u4e00-\u9fff])\.$'
punctuation_mapping = {
    ',': '，',
    '?': '？',
    '!': '！',
    ':': '：',
    ';': '；',
    '"': '”',
    "'": '’',
    '(': '（',
    ')': '）',
    '[': '【',
    ']': '】',
    '{': '｛',
    '}': '｝'
}

processed_predictions_tuned = []
processed_references_tuned = []
for pred in predictions_tuned:
    pred = pred.replace(" ", "")
    for eng_punc, chi_punc in punctuation_mapping.items():
        pred = pred.replace(eng_punc, chi_punc)
    pred = re.sub(pattern, r'\1。', pred)
    processed_predictions_tuned.append(pred)

for ref in references_tuned:
    ref = ref[0].replace(" ", "")
    for eng_punc, chi_punc in punctuation_mapping.items():
        ref = ref.replace(eng_punc, chi_punc)
    ref = re.sub(pattern, r'\1。', ref)
    processed_references_tuned.append([ref])

def process_string(text):
    return replace_punctuation_with_zh(remove_spaces(text))

def remove_spaces(text):
    return ''.join(text.split())

# Replacing en punctuation with the zh counterparts
def replace_punctuation_with_zh(text):
    # Define a mapping of English punctuation to Chinese punctuation
    punctuation_mapping = {
        ',': '，',
        '.': '。',
        '?': '？',
        '!': '！',
        ':': '：',
        ';': '；',
        '"': '”',
        "'": '’',
        '(': '（',
        ')': '）',
        '[': '【',
        ']': '】',
        '{': '｛',
        '}': '｝'
    }

    # Replace each English punctuation with its Chinese counterpart
    for eng_punc, chi_punc in punctuation_mapping.items():
        text = text.replace(eng_punc, chi_punc)

    return text

processed_predictions_tuned = [process_string(sent) for sent in predictions_tuned]
processed_references_tuned = [process_string(sent[0]) for sent in references_tuned]

bleu_score_no_smoothing = sacrebleu.corpus_bleu(processed_predictions_tuned, processed_references_tuned, tokenize='zh')
bleu_score_add_1 = sacrebleu.corpus_bleu(processed_predictions_tuned, processed_references_tuned, tokenize='zh', smooth_method="add-k", smooth_value=1)

print(f"BLEU Score without smoothing: BLEU = {bleu_score_no_smoothing}")
print(f"BLEU Score with smoothing: BLEU = {bleu_score_add_1}")

Row:  0
Translating  Last year I showed these two slides so that  demonstrate that the arctic ice cap,  which for most of the last three million years  has been the size of the lower 48 states,  has shrunk by 40 percent.
Row:  1
Translating  But this understates the seriousness of this particular problem  because it doesn't show the thickness of the ice.
Row:  2
Translating  The arctic ice cap is, in a sense,  the beating heart of the global climate system.
Row:  3
Translating  It expands in winter and contracts in summer.
Row:  4
Translating  The next slide I show you will be  a rapid fast-forward of what's happened over the last 25 years.
Row:  5
Translating  The permanent ice is marked in red.
Row:  6
Translating  As you see, it expands to the dark blue --  that's the annual ice in winter,  and it contracts in summer.
Row:  7
Translating  The so-called permanent ice, five years old or older,  you can see is almost like blood,  spilling out of the body here.
Row:  8
Translating  In 2

In [ ]:
# Prepare for backwards evaluation from chinese to english of pretrained model
prefix = 'translate to en: '

predictions_back_tuned, references_back_tuned = [], []

for index, row in val.iterrows():
    print("Row: ", str(index))
    print("Translating ", row["zh"])
    src_text = prefix + row['zh']
    input_ids = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
    generated_tokens = model.generate(input_ids)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    predictions_back_tuned.append(translation)
    references_back_tuned.append(row['en'])

references_back_tuned = [[reference] for reference in references_back_tuned]

pattern = r'([a-zA-Z])\.$'
punctuation_mapping = {
    ',': '，',
    '?': '？',
    '!': '！',
    ':': '：',
    ';': '；',
    '"': '”',
    "'": '’',
    '(': '（',
    ')': '）',
    '[': '【',
    ']': '】',
    '{': '｛',
    '}': '｝'
}

processed_predictions_back_tuned = []
processed_references_back_tuned = []
for pred in predictions_back_tuned:
    pred = pred.replace(" ", "")
    for chi_punc, eng_punc in punctuation_mapping.items():
        pred = pred.replace(chi_punc, eng_punc)
    pred = re.sub(pattern, r'\1。', pred)
    processed_predictions_back_tuned.append(pred)

for ref in references_back_tuned:
    ref = ref[0].replace(" ", "")
    for chi_punc, eng_punc in punctuation_mapping.items():
        ref = ref.replace(chi_punc, eng_punc)
    ref = re.sub(pattern, r'\1。', ref)
    processed_references_back_tuned.append([ref])

def process_string(text):
    return replace_punctuation_with_en(remove_spaces(text))

def remove_spaces(text):
    return ''.join(text.split())

# Replacing en punctuation with the zh counterparts
def replace_punctuation_with_en(text):
    # Define a mapping of English punctuation to Chinese punctuation
    punctuation_mapping = {
        '，': ',',
        '。': '.',
        '？': '?',
        '！': '!',
        '：': ':',
        '；': ';',
        '”': '"',
        '’': "'",
        '（': '(',
        '）': ')',
        '【': '[',
        '】': ']',
        '｛': '{',
        '｝': '}'
    }

    # Replace each English punctuation with its Chinese counterpart
    for chi_punc, eng_punc in punctuation_mapping.items():
        text = text.replace(chi_punc, eng_punc)

    return text

processed_predictions_back_tuned = [process_string(sent) for sent in predictions_back_tuned]
processed_references_back_tuned = [process_string(sent[0]) for sent in references_back_tuned]

bleu_score_no_smoothing_back = sacrebleu.corpus_bleu(processed_predictions_back_tuned, processed_references_back_tuned)
bleu_score_add_1_back = sacrebleu.corpus_bleu(processed_predictions_back_tuned, processed_references_back_tuned, smooth_method="add-k", smooth_value=1)

print(f"BLEU Score without smoothing: BLEU = {bleu_score_no_smoothing_back}")
print(f"BLEU Score with smoothing: BLEU = {bleu_score_add_1_back}")

Row:  0
Translating  去年我给各位展示了两个 关于北极冰帽的演示 在过去三百万年中 其面积由相当于美国南方48州面积总和 缩减了40%
Row:  1
Translating  但这些没能完全说明这个问题的严重性 因为这没有表示出冰帽的厚度
Row:  2
Translating  感觉上，北极冰帽 就好象全球气候系统中跳动的心脏
Row:  3
Translating  冬天心脏舒张，夏天心脏收缩
Row:  4
Translating  下面我要展示的是 在过去25年里的极剧变化
Row:  5
Translating  红色的是永冻冰
Row:  6
Translating  你看，它正在变成深蓝色 这是每年冬天形成的年度冰 在夏天永冻冰收缩
Row:  7
Translating  所谓的“永冻”，是指形成五年或更久的冰 你看，这也像血液一样 输送到身体各部位
Row:  8
Translating  在25年的时间里，它从这里，到了这里
Row:  9
Translating  值得注意的是 温室效应使得北冰洋周围的冻土层受热 而这里有大量被冻封的碳 解冻时，微生物降解碳形成甲烷
Row:  10
Translating  如果突破顶点，温室气体排放量 将是现有大气层中的全球温室污染总量
Row:  11
Translating  在阿拉斯加的一些浅湖里 已经可以看到水中探头的沼气泡
Row:  12
Translating  去年冬天，University of Alaska的Katey Walter教授 结队去了一个浅湖
Row:  13
Translating  哇（开心而又惊叹滴笑） 戈尔：她很好，我们怎么样呢？
Row:  14
Translating  有一个原因，北方沉积的大量热能 加热了格陵兰岛
Row:  15
Translating  这是一条每年融化的河
Row:  16
Translating  但流量却比往年都要大
Row:  17
Translating  这是格陵兰岛西南的Kangerlussuaq河
Row:  18
Translating  如果你想了解陆地上的冰块融化 如何使得海平面上升 这里就是它的入海口
Row:  19
Translating  这里的流量正在急速上升
Row:  20
Tran